In [1]:
!pip install langgraph langchain langchain-community langchain-openai openai chromadb tiktoken unstructured serpapi
!pip install chromadb llama-index llama-index-vector-stores-chroma fastembed transformers huggingface_hub torch gradio
!pip install llama-index-embeddings-fastembed
!pip install smolagents torch
!pip install pymongo dnspython google-search-results
!pip install -U llama-index
!pip install chromadb \
    llama-index \
    llama-index-vector-stores-chroma \
    llama-index-embeddings-fastembed \
    llama-index-llms-huggingface \
    fastembed \
    transformers \
    huggingface_hub

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 6.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 40.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.2/145.2 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 92.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 65.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 112.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 104.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [1]:
from smolagents import CodeAgent, GoogleSearchTool, HfApiModel, VisitWebpageTool, CodeAgent
from typing import Dict, TypedDict, List
from langgraph.graph import END, StateGraph
from llama_index.core import QueryBundle, VectorStoreIndex, StorageContext, SimpleDirectoryReader
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.core.tools import FunctionTool
from llama_index.core.agent.workflow import AgentWorkflow, ReActAgent
import chromadb
import torch
from transformers import AutoTokenizer
from llama_index.embeddings.fastembed import FastEmbedEmbedding
from llama_index.llms.huggingface import HuggingFaceLLM
from smolagents import GoogleSearchTool
import os
from pymongo import MongoClient
from datetime import datetime
import uuid
from urllib.parse import quote_plus
import json  # Added missing import
from chromadb.utils import embedding_functions
from llama_index.core.schema import TextNode

import numpy as np
import torch
from typing import TypedDict, Optional, List, Dict, Any
from langgraph.graph import StateGraph, START, END
from langchain_core.messages import HumanMessage
from langchain_openai import ChatOpenAI
from diffusers import StableDiffusionPipeline
from llama_index.core import Document
import firebase_admin
from firebase_admin import credentials, firestore

In [2]:
 # Authenticate Hugging Face (if needed)
from huggingface_hub import notebook_login
notebook_login()

In [3]:
# Initialize components
os.environ["SERPER_API_KEY"] = "2b31aea7a2305eaf1d86843679d302b736f3556d"
openrouter_api_key = "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"

In [4]:
chat_model = ChatOpenAI(
    temperature=0,
    openai_api_base="https://openrouter.ai/api/v1",
    openai_api_key=openrouter_api_key,  # Ensure this is the actual key, not a string reference
    model="mistralai/mistral-7b-instruct:v0.1"  # Or any other OpenRouter model
)


sd_pipe = StableDiffusionPipeline.from_pretrained("CompVis/stable-diffusion-v1-4", torch_dtype=torch.float16)
sd_pipe.to("cuda")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

StableDiffusionPipeline {
  "_class_name": "StableDiffusionPipeline",
  "_diffusers_version": "0.32.2",
  "_name_or_path": "CompVis/stable-diffusion-v1-4",
  "feature_extractor": [
    "transformers",
    "CLIPImageProcessor"
  ],
  "image_encoder": [
    null,
    null
  ],
  "requires_safety_checker": true,
  "safety_checker": [
    "stable_diffusion",
    "StableDiffusionSafetyChecker"
  ],
  "scheduler": [
    "diffusers",
    "PNDMScheduler"
  ],
  "text_encoder": [
    "transformers",
    "CLIPTextModel"
  ],
  "tokenizer": [
    "transformers",
    "CLIPTokenizer"
  ],
  "unet": [
    "diffusers",
    "UNet2DConditionModel"
  ],
  "vae": [
    "diffusers",
    "AutoencoderKL"
  ]
}

In [5]:
# Define all states
class RouterState(TypedDict):
    user_input: str
    request_type: Optional[str]
    messages: List[Dict[str, Any]]

class AgentState(TypedDict):
    question: str
    generation: str
    source: str
    decision_reason: str

class CombinedState(TypedDict):
    user_input: str
    request_type: Optional[str]
    messages: List[Dict[str, Any]]
    question: str
    generation: str
    source: str
    decision_reason: str

In [6]:
# Initialize LLMs
chat_model = ChatOpenAI(
    temperature=0,
    openai_api_base="https://openrouter.ai/api/v1",
    openai_api_key=openrouter_api_key,
    model="mistralai/mistral-7b-instruct:v0.1"
)

# Initialize RAG components
documents = SimpleDirectoryReader("/content/Data").load_data()
embed_model = FastEmbedEmbedding(model_name="BAAI/bge-small-en-v1.5")

tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.1")
stopping_ids = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("</s>"),
]

llm = HuggingFaceLLM(
    context_window=8192,
    max_new_tokens=256,
    generate_kwargs={"temperature": 0.7, "do_sample": True},
    system_prompt = '''
**Role**: Expert Marketing & Influencer Strategy Assistant

**Specializations**:
- Digital marketing strategies
- Influencer campaign planning
- Social media content optimization
- Small business promotion tactics

**Context Boundaries**:
✅ APPROPRIATE TOPICS:
- Influencer identification/outreach
- Content marketing strategies
- Social media campaign planning
- Audience engagement techniques
- Performance analytics interpretation

❌ OUT-OF-SCOPE TOPICS:
- Unrelated business functions (accounting, HR, etc.)
- Technical platform support
- Political/controversial subjects
- Non-marketing advice

**Response Guidelines**:
1. **Precision**: Provide specific, actionable recommendations
2. **Examples**: Include relevant case studies or templates when possible
3. **Data-Driven**: Reference metrics/KPIs where applicable
4. **Tiered Suggestions**: Offer options for different budget levels
5. **Safety**: Never recommend unethical practices

**Format Requirements**:
```markdown
📌 [Core Recommendation]
🔍 [Rationale]
💡 [Implementation Steps]
📊 [Expected Outcomes]
🌰 [Example]  ''',
    query_wrapper_prompt="[INST] {query_str} [/INST]",
    tokenizer_name="mistralai/Mistral-7B-Instruct-v0.1",
    model_name="mistralai/Mistral-7B-Instruct-v0.1",
    device_map="auto",
    stopping_ids=stopping_ids,
    tokenizer_kwargs={"max_length": 4096},
    model_kwargs={"torch_dtype": torch.float16}
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [3]:
# Initialize Firebase (add this before your ChromaDB initialization)
cred = credentials.Certificate("serviceAccountKey.json")
firebase_admin.initialize_app(cred)
db = firestore.client()

In [7]:
# Initialize ChromaDB
chroma_client = chromadb.PersistentClient(path="./chroma_db")
vector_store = ChromaVectorStore(chroma_collection=chroma_client.get_or_create_collection("my_collection"))
storage_context = StorageContext.from_defaults(vector_store=vector_store)
index = VectorStoreIndex.from_documents(
    documents,
    storage_context=storage_context,
    embed_model=embed_model
)

# Initialize Memory
memory_client = chromadb.PersistentClient(path="./memory_db")
memory_vector_store = ChromaVectorStore(
    chroma_collection=memory_client.get_or_create_collection("conversation_memory")
)
memory_storage_context = StorageContext.from_defaults(vector_store=memory_vector_store)


In [8]:
# Search Tools
def search_web(query: str) -> str:
    try:
        search_tool = GoogleSearchTool(provider="serper")
        results = search_tool(query)
        if isinstance(results, str):
            return results
        formatted = []
        for res in results[:3]:
            title = res.get('title', 'No title')
            link = res.get('link', 'No link')
            snippet = res.get('snippet', 'No snippet')
            formatted.append(f"Title: {title}\nLink: {link}\nSnippet: {snippet}")
        return "\n\n".join(formatted) if formatted else "No web results found"
    except Exception as e:
        return f"Web search error: {str(e)}"

def search_documents(query: str) -> str:
    try:
        query_engine = index.as_query_engine(llm=llm)
        response = query_engine.query(query)
        return str(response) if response else "No relevant documents found"
    except Exception as e:
        return f"Document search error: {str(e)}"


In [9]:
# Memory Functions
def check_memory(state: CombinedState) -> Dict:
    try:
        collection = memory_vector_store._collection
        embedding = embed_model.get_text_embedding(state["question"])
        results = collection.query(
            query_embeddings=[embedding],
            n_results=1,
            include=["documents", "metadatas", "distances"]
        )
        if results and results['distances'][0]:
            similarity_score = 1 - results['distances'][0][0]
            if similarity_score > 0.7:
                return {
                    "generation": results['documents'][0][0],
                    "source": "memory",
                    "decision_reason": f"Memory match (similarity: {similarity_score:.2f})",
                    "question": state["question"]
                }
    except Exception as e:
        print(f"Memory check error: {str(e)}")
    return state

def store_in_memory(state: CombinedState) -> Dict:
    if state["source"] in ["documents", "web"]:
        try:
            chroma_collection = memory_vector_store._collection
            embedding = embed_model.get_text_embedding(state["question"])
            metadata = {
                "question": state["question"],
                "source": state["source"],
                "timestamp": datetime.now().isoformat()
            }
            chroma_collection.add(
                embeddings=[embedding],
                documents=[state["generation"]],
                metadatas=[metadata],
                ids=[str(uuid.uuid4())]
            )
        except Exception as e:
            print(f"Memory storage error: {str(e)}")
    return state

In [16]:
# # First, update your ToolDecisionAgent rules
# class ToolDecisionAgent:
#     def __init__(self, llm):
#         self.llm = llm
#         self.rules = """Decision Rules:
#         1. Use documents for relationship marketing questions
#         2. Use web for current events/trends
#         3. Use influencers when asking about:
#            - Influencer recommendations
#            - Social media personalities
#            - Creator marketing
#            - Brand collaborations
#         4. Default to documents if available"""

#     def decide_tool(self, question: str) -> Dict:
#         prompt = f"""Analyze this question and follow these rules:
#         {self.rules}
#         Question: {question}
#         Return JSON: {{"tool": "documents"|"web"|"influencers", "reason": "explanation"}}"""
#         try:
#             response = self.llm.complete(prompt)
#             return json.loads(response.text)
#         except:
#             return {"tool": "documents", "reason": "default fallback"}

# # Then update the execute_search function
# def execute_search(state: CombinedState) -> Dict:
#     try:
#         if state["source"] == "documents":
#             result = search_documents(state["question"])
#             if "no relevant" in result.lower():
#                 result = search_web(state["question"])
#                 state["source"] = "web"
#         elif state["source"] == "web":
#             result = search_web(state["question"])
#         elif state["source"] == "influencers":
#             result = search_influencers(state)
#         else:
#             result = "No valid search tool selected"
#         return {**state, "generation": result}
#     except Exception as e:
#         return {**state, "generation": f"Search error: {str(e)}"}

# # Add the influencer search function
# def search_influencers(state: CombinedState) -> str:
#     """Search influencers based on the question and return formatted results"""
#     question = state["question"].lower()

#     # Extract search parameters from question
#     country = None
#     if "tunisia" in question:
#         country = "tunisia"
#     elif "morocco" in question:
#         country = "morocco"

#     topic = None
#     for t in ["beauty", "fashion", "technology", "lifestyle"]:
#         if t in question:
#             topic = t
#             break

#     min_followers = 0
#     if "macro" in question:
#         min_followers = 100000  # 100K for macro-influencers
#     elif "micro" in question:
#         min_followers = 10000   # 10K for micro-influencers

#     # Fetch from Firebase
#     influencers = get_influencers_from_firebase(
#         country_filter=country,
#         topic_filter=topic,
#         min_followers=min_followers,
#         limit=10  # Get top 10 by followers
#     )

#     if not influencers:
#         return "No influencers found matching these criteria"

#     # Format results
#     results = ["🎯 Influencer Search Results:"]
#     for inf in influencers:
#         results.append(
#             f"🌟 {inf['name']} (@{inf['handle']})\n"
#             f"👥 {inf['followers']} followers | 📍 {inf['country'].title()}\n"
#             f"🏷️ Topic: {inf['topics']}\n"
#             f"📸 {inf['instagram_url']}\n"
#             "―" * 40
#         )

#     return "\n".join(results)

# # Helper function (updated for your schema)
# def get_influencers_from_firebase(
#     country_filter: Optional[str] = None,
#     topic_filter: Optional[str] = None,
#     min_followers: int = 0,
#     limit: int = 10
# ) -> List[Dict]:
#     """Fetch influencers from Firebase with your exact schema"""
#     if not firebase_admin._apps:
#         cred = credentials.Certificate("serviceAccountKey.json")
#         firebase_admin.initialize_app(cred)

#     db = firestore.client()
#     collection_ref = db.collection("InfluencerMarketMind")

#     # Apply filters
#     if country_filter:
#         collection_ref = collection_ref.where("country", "==", country_filter.lower())
#     if topic_filter:
#         collection_ref = collection_ref.where("Topics", "==", topic_filter.lower())

#     influencers = []

#     try:
#         docs = collection_ref.limit(limit).stream()
#         for doc in docs:
#             data = doc.to_dict()
#             followers_str = data.get("Followers", "0")
#             followers_num = parse_followers(followers_str)

#             if followers_num >= min_followers:
#                 influencers.append({
#                     "id": doc.id,
#                     "name": data.get("Nom", "").strip(),
#                     "handle": data.get("IG_Handle", "").strip().lower(),
#                     "followers": followers_str,
#                     "followers_num": followers_num,
#                     "topics": data.get("Topics", "").strip(),
#                     "country": data.get("country", "").strip().lower(),
#                     "instagram_url": data.get("Lien Instagram", "").strip()
#                 })

#         # Sort by follower count
#         influencers.sort(key=lambda x: x["followers_num"], reverse=True)
#         return influencers

#     except Exception as e:
#         print(f"Firebase error: {str(e)}")
#         return []

# def parse_followers(followers_str: str) -> int:
#     """Convert formatted follower string to integer"""
#     try:
#         if 'K' in followers_str:
#             return int(float(followers_str.replace('K', '')) * 1000)
#         if 'M' in followers_str:
#             return int(float(followers_str.replace('M', '')) * 1000000)
#         return int(followers_str)
#     except:
#         return 0

In [25]:
# Add the influencer search function
def search_influencers(state: CombinedState) -> str:
    """Search influencers based on the question and return formatted results"""
    question = state["question"].lower()

    # Extract search parameters from question
    country = None
    if "tunisia" in question:
        country = "tunisia"
    elif "morocco" in question:
        country = "morocco"

    topic = None
    for t in ["beauty", "fashion", "technology", "lifestyle"]:
        if t in question:
            topic = t
            break

    min_followers = 0
    if "macro" in question:
        min_followers = 100000  # 100K for macro-influencers
    elif "micro" in question:
        min_followers = 10000   # 10K for micro-influencers

    # Fetch from Firebase
    influencers = get_influencers_from_firebase(
        country_filter=country,
        topic_filter=topic,
        min_followers=min_followers,
        limit=10  # Get top 10 by followers
    )

    if not influencers:
        return "No influencers found matching these criteria"

    # Format results
    results = ["🎯 Influencer Search Results:"]
    for inf in influencers:
        results.append(
            f"🌟 {inf['name']} (@{inf['handle']})\n"
            f"👥 {inf['followers']} followers | 📍 {inf['country'].title()}\n"
            f"🏷️ Topic: {inf['topics']}\n"
            f"📸 {inf['instagram_url']}\n"
            "―" * 40
        )

    return "\n".join(results)

# Helper function (updated for your schema)
def get_influencers_from_firebase(
    country_filter: Optional[str] = None,
    topic_filter: Optional[str] = None,
    min_followers: int = 0,
    limit: int = 10
) -> List[Dict]:
    """Fetch influencers from Firebase with your exact schema"""
    if not firebase_admin._apps:
        cred = credentials.Certificate("serviceAccountKey.json")
        firebase_admin.initialize_app(cred)

    db = firestore.client()
    collection_ref = db.collection("InfluencerMarketMind")

    # Apply filters
    if country_filter:
        collection_ref = collection_ref.where("country", "==", country_filter.lower())
    if topic_filter:
        collection_ref = collection_ref.where("Topics", "==", topic_filter.lower())

    influencers = []

    try:
        docs = collection_ref.limit(limit).stream()
        for doc in docs:
            data = doc.to_dict()
            followers_str = data.get("Followers", "0")
            followers_num = parse_followers(followers_str)

            if followers_num >= min_followers:
                influencers.append({
                    "id": doc.id,
                    "name": data.get("Nom", "").strip(),
                    "handle": data.get("IG_Handle", "").strip().lower(),
                    "followers": followers_str,
                    "followers_num": followers_num,
                    "topics": data.get("Topics", "").strip(),
                    "country": data.get("country", "").strip().lower(),
                    "instagram_url": data.get("Lien Instagram", "").strip()
                })

        # Sort by follower count
        influencers.sort(key=lambda x: x["followers_num"], reverse=True)
        return influencers

    except Exception as e:
        print(f"Firebase error: {str(e)}")
        return []

def parse_followers(followers_str: str) -> int:
    """Convert formatted follower string to integer"""
    try:
        if 'K' in followers_str:
            return int(float(followers_str.replace('K', '')) * 1000)
        if 'M' in followers_str:
            return int(float(followers_str.replace('M', '')) * 1000000)
        return int(followers_str)
    except:
        return 0

In [14]:
# Decision Agent
class ToolDecisionAgent:
    def __init__(self, llm):
        self.llm = llm
        self.rules = self.rules = """**Tool Selection Framework**

1. 📂 DOCUMENTS (Knowledge Base):
   - Relationship marketing strategies
   - Influencer identification criteria
   - Campaign performance benchmarks
   - Historical campaign data
   - Specific creator recommendations
   - Audience demographic matching
   - Follower count filters
   - Geo-targeted influencer searches
   *Example queries*:
   "Find fashion influencers in Morocco with 50K-100K followers"
   "Micro-influencers for eco-friendly products"
   - Template/playbook requests
   *Example queries*:
   "How to build influencer relationships?"
   "Show me beauty influencer case studies"

2. 🌐 WEB (Live Search):
   - Breaking industry news
   - Platform algorithm changes
   - Emerging social media trends
   - Real-time event coverage
   *Example queries*:
   "Latest Instagram Reels trends 2024"
   "TikTok ban updates today"

4. 🛠️ DEFAULT HIERARCHY:
   Documents → Influencers → Web
   (Always check knowledge base first,
   then influencer database,
   then web as last resort)

**Conflict Resolution**:
- If query contains both trend + influencer terms → Prioritize INFLUENCERS
- For metric comparisons → DOCUMENTS + WEB combo
- When uncertain → Follow default hierarchy

**Output Format**:
```json
{
  "tool": "selected_source",
  "confidence": 0-100,
  "fallback": "alternate_source",
  "rationale": "matching_criteria"
}
```"""

    def decide_tool(self, question: str) -> Dict:
        prompt = f"""Analyze this question and follow these rules:
        {self.rules}
        Question: {question}
        Return JSON: {{"tool": "documents"|"web", "reason": "explanation"}}"""
        try:
            response = self.llm.complete(prompt)
            return json.loads(response.text)
        except:
            return {"tool": "documents", "reason": "default fallback"}

def decide_tool(state: CombinedState) -> Dict:
    decision_agent = ToolDecisionAgent(llm)
    decision = decision_agent.decide_tool(state["question"])
    return {
        **state,
        "source": decision["tool"],
        "decision_reason": decision["reason"]
    }

def execute_search(state: CombinedState) -> Dict:
    try:
        if state["source"] == "documents":
            result = search_documents(state["question"])
            if "no relevant" in result.lower():
                result = search_web(state["question"])
                state["source"] = "web"
        else:
            result = search_web(state["question"])
        return {**state, "generation": result}
    except Exception as e:
        return {**state, "generation": f"Search error: {str(e)}"}


In [15]:
def classify_request(state: RouterState) -> Dict:
    """Advanced request classifier with multi-layered validation and fallback handling"""

    # Optimized prompt with structured examples and constraints
    prompt = f"""**Marketing Content Classification Task**

**Instruction**:
Analyze the user's request and classify it EXCLUSIVELY as one of these options:
1. "text" - For written content needs (captions, posts, articles)
2. "image" - For visual content requests (graphics, photos, designs)
3. "both" - For combined text+visual projects

**Decision Framework**:
- Text Indicators: "write", "caption", "post", "blog", "content", "text"
- Visual Indicators: "design", "visual", "photo", "image", "graphic"
- Combined Indicators: "package", "both", "complete content"

**Examples**:
1. "Help me write a product description" → "text"
2. "Create social media images" → "image"
3. "Need a complete Instagram post" → "both"

**Strict Requirements**:
- Respond ONLY with the classification token
- No explanations or additional text
- Default to "text" if uncertain

**Request to Classify**:
{state['user_input']}"""

    try:
        # First attempt with primary model
        response = chat_model.invoke([HumanMessage(content=prompt)])
        clean_response = response.content.strip().lower()

        # Multi-stage validation
        classification = None
        for token in ["text", "image", "both"]:
            if token in clean_response:  # Flexible matching
                classification = token
                break

        if not classification:
            # Fallback to secondary validation
            clean_response = (
                clean_response.replace('"', '')
                .replace("'", "")
                .replace(".", "")
                .strip()
            )
            classification = clean_response if clean_response in ["text", "image", "both"] else "text"

        # Confidence scoring
        confidence = "high"
        if classification == "text" and any(word in state['user_input'].lower() for word in ["design", "visual"]):
            confidence = "low"

        return {
            "request_type": classification,
            "classification_confidence": confidence,
            "messages": state["messages"] + [
                {
                    "role": "system",
                    "content": f"Classification: {classification} (Confidence: {confidence})"
                }
            ]
        }

    except Exception as e:
        # Comprehensive fallback handling
        error_id = str(uuid.uuid4())[:8]
        print(f"Classification Error [{error_id}]: {str(e)}")

        return {
            "request_type": "text",
            "classification_confidence": "unknown",
            "error_id": error_id,
            "messages": state["messages"] + [
                {
                    "role": "system",
                    "content": f"Classification failed. Error ID: {error_id}"
                }
            ]
        }

In [16]:
# Workflows
# RAG Workflow
rag_workflow = StateGraph(AgentState)
rag_workflow.add_node("check_memory", check_memory)
rag_workflow.add_node("decide_tool", decide_tool)
rag_workflow.add_node("execute_search", execute_search)
rag_workflow.add_node("store_memory", store_in_memory)
rag_workflow.set_entry_point("check_memory")
rag_workflow.add_conditional_edges(
    "check_memory",
    lambda x: "store_memory" if x.get("generation") else "decide_tool"
)
rag_workflow.add_edge("decide_tool", "execute_search")
rag_workflow.add_edge("execute_search", "store_memory")
rag_workflow.add_edge("store_memory", END)
rag_app = rag_workflow.compile()

# Router Workflow
router_workflow = StateGraph(RouterState)
router_workflow.add_node("classify_request", classify_request)
router_workflow.set_entry_point("classify_request")
router_workflow.add_conditional_edges(
    "classify_request",
    lambda x: x["request_type"],
    {"text": END, "image": END, "both": END}
)
router_app = router_workflow.compile()


In [ ]:
def run_agent():
    print("Marketing AI Assistant Initialized")
    print(f"Documents: {'Loaded' if documents else 'Not available'}")

    while True:
        try:
            user_input = input("\nQuestion (or 'exit' to quit): ").strip()

            if user_input.lower() in ['exit', 'quit', 'bye']:
                print("Goodbye!")
                break

            if not user_input:
                continue

            # First try memory
            memory_response = check_memory({
                "question": user_input,
                "generation": "",
                "source": "",
                "decision_reason": "",
                "user_input": user_input,
                "request_type": None,
                "messages": []
            })

            if memory_response.get("generation"):
                print(f"\nAnswer (from memory):\n{memory_response['generation']}")
                continue

            # Classify request with error handling
            try:
                classification = classify_request({
                    "user_input": user_input,
                    "request_type": None,
                    "messages": []
                })
                req_type = classification["request_type"]
            except:
                req_type = "text"

            # Handle each request type with fallbacks
            if req_type == "text":
                try:
                    result = rag_app.invoke({
                        "question": user_input,
                        "generation": "",
                        "source": "",
                        "decision_reason": ""
                    })
                    print(f"\nMarketing Answer:\n{result['generation']}")
                except:
                    print("\nHere's information about marketing strategies...")

            elif req_type == "image":
                print("\nFor marketing visuals, I recommend:")
                print("- Canva: canva.com")
                print("- Adobe Express: adobe.com/express")

            elif req_type == "both":
                try:
                    result = rag_app.invoke({
                        "question": user_input,
                        "generation": "",
                        "source": "",
                        "decision_reason": ""
                    })
                    print(f"\nMarketing Content:\n{result['generation']}")
                    print("\nVisual Suggestions:")
                    print("- Create an infographic with key points")
                    print("- Design social media posts")
                except:
                    print("\nMarketing advice with visual recommendations...")

        except Exception as e:
            print("\nLet me provide some general marketing guidance...")
            print("Relationship marketing focuses on customer retention...")

if __name__ == "__main__":
    run_agent()

Marketing AI Assistant Initialized
Documents: Loaded

Question (or 'exit' to quit): suggest me tunisian influencers to promote my beauty products business. Give me their instagram followers as well.


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



Marketing Answer:
📌 Recommendation:
- Identify influencers with a strong presence in Tunisia and a substantial following in the beauty niche.
- Focus on influencers who have an engaged audience and can effectively promote your brand.

🔍 Rationale:
- Tunisian influencers can help you reach a local market and increase brand awareness.
- A targeted approach will help you connect with the right audience and achieve better results.

💡 Implementation Steps:
1. Research popular beauty influencers in Tunisia on Instagram.
2. Look for influencers who have a significant following and are actively engaging with their audience.
3. Analyze their content to ensure it aligns with your brand values and target audience.
4. Reach out to the influencers and discuss potential collaboration opportunities.

📊 Expected Outcomes:
- Increased brand visibility and recognition in Tunisia.
- Enhanced product exposure to a targeted audience.
- Improved sales and customer engagement.

🌰 Example:
- Collaborate with

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



Marketing Answer:
📌 Suggest a list of Moroccan beauty influencers and their Instagram usernames.

🔍 Conduct thorough research on popular Moroccan beauty influencers.

💡 Compile a list of influencers with a significant following in the beauty niche, focusing on those based in Morocco.

📊 Provide the Instagram usernames of the influencers to ensure a more accurate search.

🌰 Include well-known influencers such as Nadia Zahra @nadiazahra, Fatima Zahra @fatimazahran, and Sanae El Allami @sanaeelallami.


In [20]:
def debug_memory():
    """Print current memory contents"""
    collection = memory_vector_store._collection
    results = collection.peek()
    print("\nMemory Contents:")
    for doc, meta in zip(results['documents'], results['metadatas']):
        print(f"Question: {meta['question']}")
        print(f"Answer: {doc[:100]}...")
        print("-----")

In [21]:
debug_memory()


Memory Contents:


In [18]:
def clear_memory() -> bool:
    """
    Clears all conversation memory from ChromaDB
    Returns:
        bool: True if successful, False if error occurred
    """
    try:
        # Get the memory collection
        memory_collection = memory_client.get_collection("conversation_memory")

        # Get all document IDs
        ids = memory_collection.get()["ids"]

        # Delete all documents
        if ids:
            memory_collection.delete(ids=ids)

        print("🗑️ Successfully cleared all conversation memory")
        return True

    except Exception as e:
        print(f"❌ Error clearing memory: {str(e)}")
        return False

In [19]:
clear_memory()

🗑️ Successfully cleared all conversation memory


True

In [6]:
import firebase_admin
from firebase_admin import credentials, firestore
from typing import List, Dict, Optional

def get_influencers_from_firebase(
    collection_name: str = "InfluencerMarketMind",
    country_filter: Optional[str] = None,
    topic_filter: Optional[str] = None,
    min_followers: int = 0,
    limit: int = 1000
) -> List[Dict]:
    """
    Fetches influencer data from Firebase Firestore with your exact schema.

    Args:
        collection_name: Name of your Firestore collection
        country_filter: Filter by country (e.g. "morocco")
        topic_filter: Filter by exact topic (e.g. "technology")
        min_followers: Minimum follower count (converts K/M suffixes to numbers)
        limit: Maximum number of records to return

    Returns:
        List of influencer dictionaries with these fields:
        - name: Full name
        - handle: Instagram handle
        - followers: Formatted string (e.g. "11.9K")
        - followers_num: Numeric value for sorting
        - topics: Single topic string
        - country: Lowercase country code
        - instagram_url: Full Instagram URL
    """
    # Initialize Firebase if not already done
    if not firebase_admin._apps:
        cred = credentials.Certificate("serviceAccountKey.json")
        firebase_admin.initialize_app(cred)

    db = firestore.client()
    collection_ref = db.collection(collection_name)

    # Apply filters
    if country_filter:
        collection_ref = collection_ref.where("country", "==", country_filter.lower())

    if topic_filter:
        collection_ref = collection_ref.where("Topics", "==", topic_filter.lower())

    influencers = []

    try:
        # Get documents with limit
        docs = collection_ref.limit(limit).stream()

        for doc in docs:
            data = doc.to_dict()
            followers_str = data.get("Followers", "0")

            # Convert followers to numeric value
            followers_num = parse_followers(followers_str)

            # Skip if doesn't meet follower threshold
            if followers_num < min_followers:
                continue

            influencer = {
                "id": doc.id,
                "name": data.get("Nom", "").strip(),
                "handle": data.get("IG_Handle", "").strip().lower(),
                "followers": followers_str,
                "followers_num": followers_num,
                "topics": data.get("Topics", "").strip().lower(),
                "country": data.get("country", "").strip().lower(),
                "instagram_url": data.get("Lien Instagram", "").strip(),
                "raw_data": data  # Include all original data
            }
            influencers.append(influencer)

    except Exception as e:
        print(f"Firebase query error: {str(e)}")

    # Sort by follower count descending
    influencers.sort(key=lambda x: x["followers_num"], reverse=True)

    return influencers

def parse_followers(followers_str: str) -> int:
    """Convert formatted follower string to integer"""
    try:
        if 'K' in followers_str:
            return int(float(followers_str.replace('K', '')) * 1000)
        if 'M' in followers_str:
            return int(float(followers_str.replace('M', '')) * 1000000)
        return int(followers_str)
    except:
        return 0

In [7]:
# Get all Moroccan influencers
moroccan_influencers = get_influencers_from_firebase(country_filter="morocco")

# Get technology influencers with at least 10K followers
tech_influencers = get_influencers_from_firebase(
    topic_filter="technology",
    min_followers=10000
)

# Get top 50 influencers by follower count
top_influencers = get_influencers_from_firebase(limit=50)

/usr/local/lib/python3.11/dist-packages/google/cloud/firestore_v1/base_collection.py:303: UserWarning: Detected filter using positional arguments. Prefer using the 'filter' keyword argument instead.
  return query.where(field_path, op_string, value)


In [9]:
moroccan_influencers

[{'id': 'fg9nufAPeOQLbC6mTO90',
  'name': 'Koi Grand',
  'handle': '@koigrand',
  'followers': '868.4K',
  'followers_num': 868400,
  'topics': 'fashion',
  'country': 'morocco',
  'instagram_url': 'https://www.instagram.com/koigrand',
  'raw_data': {'country': 'morocco',
   'Followers': '868.4K',
   'Topics': 'fashion',
   'Nom': 'Koi Grand',
   'Lien Instagram': 'https://www.instagram.com/koigrand',
   'IG_Handle': '@koigrand'}},
 {'id': 'BFzWGehOaeqpazS3uwhE',
  'name': 'ʟᴏᴜʙɴᴀ ʙᴀɢʜᴅᴀᴅ',
  'handle': '@loubnabaghdad',
  'followers': '844.6K',
  'followers_num': 844600,
  'topics': 'beauty',
  'country': 'morocco',
  'instagram_url': 'https://www.instagram.com/loubnabaghdad',
  'raw_data': {'country': 'morocco',
   'Followers': '844.6K',
   'Topics': 'beauty',
   'Nom': 'ʟᴏᴜʙɴᴀ ʙᴀɢʜᴅᴀᴅ',
   'Lien Instagram': 'https://www.instagram.com/loubnabaghdad',
   'IG_Handle': '@loubnabaghdad'}},
 {'id': 'UsKyiEan8oL6oN7kIX6K',
  'name': 'Afrah chiadli',
  'handle': '@afrah_giiordaniiofficiel',
